In [2]:
import numpy as np
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

Reproduce = 2023

In [3]:
# Return the inverse demand for player 1 and 2 in a tuple
def P(q1, q2, Theta1, Theta2, Xi=6, Mu=1, Lambda=1):
    p1 = Xi - Mu * (q1 + q2) + Lambda * (Theta2 - Theta1)
    p2 = Xi - Mu * (q1 + q2) + Lambda * (Theta1 - Theta2)
    return (p1, p2)

# Return the profit for player 1 and 2 in a tuple (no costs)
def Pi(q1, q2, Theta1, Theta2, Phi, Xi=6, Mu=1, Lambda=1):
    p1, p2 = P(q1, q2, Theta1, Theta2, Xi, Mu, Lambda)
    c1, c2 = Phi * (1-Theta1)**2, Phi * (1-Theta2)**2
    return (q1*p1-c1, q2*p2-c2)

In [15]:
# Gets the next action the current player will take (epsilon greedy)
# Note: this action is the next state for the other player
def get_next_state(Q, state, actions, epsilon):
    random_action = np.random.choice(actions)
    greedy_action = max(Q[state], key = Q[state].get)
    return np.random.choice([random_action, greedy_action], p = [epsilon, 1 - epsilon])

def Qlearn(k1=4, k2=4, qc=1.5, qn=2, alpha=6, gamma=0.9, T=500_000, L=1000):
    # Given k1, k2 generate the possible actions
    start_q = ((1+k1)*qc - qn) / k1
    step_q = (qn-qc) / k1
    Qs = np.array([start_q + step_q * i for i in range(k1+3)])
    Thetas = np.array([theta/k2 for theta in range(k2+1)])
    Q_mesh, Theta_mesh = np.meshgrid(Qs, Thetas)
    actions_h = np.column_stack((Q_mesh.ravel(), Theta_mesh.ravel()))
    actions = np.array([tuple(action) for action in actions_h], dtype=[('Q', float), ('Theta', float)])

    # Pick the first to prices of both players randomly
    Action1, Action2, Action1_next, Action2_next = np.random.choice(actions, size=4)
    
    # Initialize the Q matrix for both players
    Q1 = defaultdict(lambda: dict(zip(actions, np.zeros(len(actions)))))
    Q2 = defaultdict(lambda: dict(zip(actions, np.zeros(len(actions)))))
    
    # Initialize Epsilon
    Epsilon = 1

    # Keep track of metrics
    prof_p1_ep, prof_p2_ep = [], []
    q1, q2 = [], []
    theta1, theta2 = [], []

    for t in range(3, T+1):
        # Player 1's turn
        # Calculate the current profit
        prof_p1, prof_p2 = Pi(Action1_next, Action2)

        # Add q and theta
        q1.append(Action1_next[0])
        theta1.append(Action1_next[1])

        # Get the v-value of the next state
        v = max(Q1[Action2_next].values())
        
        # Update Q1 (state X action)
        Q1[Action2][Action1_next] += alpha * (prof_p1 + gamma * Pi(Action1_next, Action2_next)[0] + gamma**2 * v - Q1[Action2][Action1_next])

        # Update current action player 1, and determine next action based on the next state
        Action1, Action1_next = Action1_next, get_next_state(Q1, Action2_next, actions, Epsilon)

        # Append the profits to the lists of profits
        prof_p1_ep.append(prof_p1)
        prof_p2_ep.append(prof_p2)

        # Player 2's turn
        # Calculate the current profit
        prof_p1, prof_p2 = Pi(Action1, Action2_next)

        # Add market price to list
        # Add q and theta
        q2.append(Action2_next[0])
        theta2.append(Action2_next[1])

        # Get the v-value of the next state
        v = max(Q2[Action1_next].values())
        
        # Update Q2 (state X action)
        Q2[Action1][Action2_next] += alpha * (prof_p2 + gamma * Pi(Action1_next, Action2_next)[1] + gamma**2 * v - Q2[Action1][Action2_next])

        # Update current action player 1, and determine next action based on the next state
        Action2, Action2_next = Action2_next, get_next_state(Q2, Action1_next, actions, Epsilon)
        
        # Append the profits to the lists of profits
        prof_p1_ep.append(prof_p1)
        prof_p2_ep.append(prof_p2)
 
        # Update epsilon
        Epsilon = 0.1**(4*t/T)
    
    # Compute average profitability of the last evaluation period runs
    average_pi1 = sum(prof_p1_ep[-L:]) / L
    average_pi2 = sum(prof_p2_ep[-L:]) / L

    # Compute average thetas of the last evaluation period runs
    average_theta1 = sum(theta1[-L:]) / L
    average_theta2 = sum(theta2[-L:]) / L

    # Get the last 50 q's
    last50_q1 = q1[-50:]
    last50_q2 = q2[-50:]
    
    return (average_pi1, average_pi2, average_theta1, average_theta2, last50_q1, last50_q2)